In [ ]:
# %%capture
# !pip install transformers wandb ml_collections
# !wandb login 8c920c1bbdcc7ee5353aaa8becd54a4942f0fc06
# !pip install timm

In [ ]:
# !touch /content/drive/MyDrive/floodnet_features/question_to_file.json


In [ ]:
# import os

# len(os.listdir("/content/drive/MyDrive/floodnet_data/Images/Valid_Image")) # 1448, 450, 450

In [ ]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


In [ ]:
import json
d = json.load(open("/content/drive/MyDrive/floodnet_data/Questions/Training Question.json", "r"))

uniq = []
for key, val in d.items():
    if val["Question"] not in uniq:
        uniq.append(val["Question"])

print(len(uniq))

# uniq = []
# for key, val in d.items():
#     if val["Question_Type"] not in uniq:
#         uniq.append(val["Question_Type"])

# print(len(uniq))

15


In [ ]:
uniq_qs = {idx:uniq[idx] for idx in range(len(uniq))}

In [ ]:
uniq_qs

{0: 'What is the overall condition of the given image?',
 1: 'How many non flooded buildings can be seen in this image?',
 2: 'How many buildings can be seen in the image?',
 3: 'How many buildings can be seen in this image?',
 4: 'Is the entire road non flooded?',
 5: 'What is the condition of the road in this image?',
 6: 'How many buildings are non flooded?',
 7: 'Is the entire road flooded?',
 8: 'How many buildings are in this image?',
 9: 'What is the condition of road?',
 10: 'How many buildings are non flooded in this image?',
 11: 'How many buildings are in the image?',
 12: 'How many buildings are flooded in this image?',
 13: 'How many buildings are flooded?',
 14: 'How many flooded buildings can be seen in this image?'}

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
from transformers import RobertaTokenizer, RobertaModel
import torch

tokenizer = RobertaTokenizer.from_pretrained("roberta-large")
model = RobertaModel.from_pretrained("roberta-large")

model.to(torch.device("cuda"))

feats = dict()

for key, value in uniq_qs.items():
    inputs = tokenizer(value, return_tensors="pt")
    inputs = {k:v.to(torch.device("cuda")) for k,v in inputs.items()}
    outputs = model(**inputs)

    last_hidden_states = outputs[1]
    feats[key] = last_hidden_states.detach().cpu()


Downloading:   0%|          | 0.00/878k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/446k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/482 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.33G [00:00<?, ?B/s]

Some weights of the model checkpoint at roberta-large were not used when initializing RobertaModel: ['lm_head.layer_norm.bias', 'lm_head.dense.bias', 'lm_head.bias', 'lm_head.decoder.weight', 'lm_head.layer_norm.weight', 'lm_head.dense.weight']
- This IS expected if you are initializing RobertaModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [ ]:
for idx, q in enumerate(uniq_qs.values()):
    savepath = "/content/drive/MyDrive/floodnet_convnext_features/Questions/" + q + ".pt"
    torch.save(feats[idx], savepath)

In [ ]:
# print(uniq)

['What is the overall condition of the given image?', 'How many non flooded buildings can be seen in this image?', 'How many buildings can be seen in the image?', 'How many buildings can be seen in this image?', 'Is the entire road non flooded?', 'What is the condition of the road in this image?', 'How many buildings are non flooded?', 'Is the entire road flooded?', 'How many buildings are in this image?', 'What is the condition of road?', 'How many buildings are non flooded in this image?', 'How many buildings are in the image?', 'How many buildings are flooded in this image?', 'How many buildings are flooded?', 'How many flooded buildings can be seen in this image?']


In [ ]:
from IPython.display import clear_output, Image, display
import PIL.Image
import io
import json
import torch
import numpy as np
import pickle
import os
# import timm
from torch.utils.data import Dataset, DataLoader
import json
import os
from glob import glob
from torchvision import transforms as T
from torchvision import transforms
from PIL import Image

from torch import nn
from tqdm import tqdm

# from transformers import AutoTokenizer, AutoModel

In [ ]:
import glob

all_train_ims = glob.glob("/content/drive/MyDrive/floodnet_data/Images/Train_Image/*")

In [ ]:
class ImageDataset(Dataset):
    def __init__(self, all_imagepaths):
        self.imgpaths = all_imagepaths
        self.tr = transforms.Compose([
                    transforms.ToTensor(),
                    transforms.Resize([224,224]),
                    transforms.Normalize(mean=[0.485, 0.456, 0.406],
                                                std=[0.229, 0.224, 0.225])
                ])
    
    def __len__(self):
        return len(self.imgpaths)
    
    def __getitem__(self, idx):
        img = self.tr(Image.open(self.imgpaths[idx]))
        return img, self.imgpaths[idx]

ds = ImageDataset(all_train_ims)
dl = DataLoader(ds, batch_size=16, num_workers=2)

In [ ]:
class ImageTextDataset(Dataset):
    def __init__(self, all_imagepaths):
        self.imgpaths = all_imagepaths
        self.tr = transforms.Compose([
                    transforms.ToTensor(),
                    transforms.Resize([224,224]),
                    transforms.Normalize(mean=[0.485, 0.456, 0.406],
                                                std=[0.229, 0.224, 0.225])
                ])
    
    def __len__(self):
        return len(self.imgpaths)
    
    def __getitem__(self, idx):
        img = self.tr(Image.open(self.imgpaths[idx]))
        return img, self.imgpaths[idx]

ds = ImageDataset(all_train_ims)
dl = DataLoader(ds, batch_size=16, num_workers=2)

In [ ]:
for batch in dl:
    print(batch)
    break

In [ ]:
import torch, gc
del resnet
gc.collect()
torch.cuda.empty_cache()

In [ ]:
!pip install ftfy regex tqdm
!pip install git+https://github.com/openai/CLIP.git

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 53 kB 1.5 MB/s 
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Cloning https://github.com/openai/CLIP.git to /tmp/pip-req-build-5hshhxqq
  Running command git clone -q https://github.com/openai/CLIP.git /tmp/pip-req-build-5hshhxqq
  Created wheel for clip: filename=clip-1.0-py3-none-any.whl size=1369387 sha256=bc913d0caa3bce2ce107e63125db9e703b5a5fffa755566c9cec6653cd6e0841
  Stored in directory: /tmp/pip-ephem-wheel-cache-lrbfu7ch/wheels/fd/b9/c3/5b4470e35ed76e174bff77c92f91da82098d5e35fd5bc8cdac
Successfully built clip


In [ ]:
# resnet = timm.create_model('resnet50', pretrained=True, num_classes=0)
# from torchvision import transforms, models

# resnet = models.vit_l_32(pretrained=True)
# list_of_layers = list(resnet.children())[:-1]
# list_of_layers.append(torch.nn.AdaptiveMaxPool2d(1))
# list_of_layers.append(torch.nn.Flatten())
# # list_of_layers.append(torch.nn.LayerNorm((1536,), eps=1e-06, elementwise_affine=True))
# resnet = torch.nn.Sequential(*list_of_layers)
# # flatten = torch.nn.Flatten()
# # norm = torch.nn.LayerNorm((768,), eps=1e-06, elementwise_affine=True)
# resnet.eval()

import torch
import clip
from PIL import Image

# device = "cuda" if torch.cuda.is_available() else "cpu"
device = torch.device("cuda")
model, preprocess = clip.load("ViT-L/14", device="cuda")

# image = preprocess(Image.open("CLIP.png")).unsqueeze(0).to(device)
# text = clip.tokenize(["a diagram", "a dog", "a cat"]).to(device)



resnet = resnet.to(device)

def create_image_features(dl, savedir):
    for batch in tqdm(dl):
        img = batch[0]
        paths = batch[1]
        img = img.to(device)
        features = resnet(img).detach().cpu()

        for idx in range(len(paths)):
            savepath = os.path.join(savedir, paths[idx].split("/")[-1].split(".")[0] + ".pt")
            torch.save(features[idx], savepath)

In [ ]:
create_image_features(dl, "/content/drive/MyDrive/floodnet_vit_features/Images/Train_Image")

  0%|          | 0/91 [00:14<?, ?it/s]


AssertionError: ignored

In [ ]:
# !rm -rf /content/drive/MyDrive/floodnet_convnext_features/Images/Train_Image
!mkdir /content/drive/MyDrive/floodnet_vit_features
!mkdir /content/drive/MyDrive/floodnet_vit_features/Images
!mkdir /content/drive/MyDrive/floodnet_vit_features/Images/Train_Image

In [ ]:
!ls /content/drive/MyDrive/floodnet_vit_features/Questions

'How many buildings are flooded in this image_.pt'
'How many buildings are flooded_.pt'
'How many buildings are in the image_.pt'
'How many buildings are in this image_.pt'
'How many buildings are non flooded in this image_.pt'
'How many buildings are non flooded_.pt'
'How many buildings can be seen in the image_.pt'
'How many buildings can be seen in this image_.pt'
'How many flooded buildings can be seen in this image_.pt'
'How many non flooded buildings can be seen in this image_.pt'
'Is the entire road flooded_.pt'
'Is the entire road non flooded_.pt'
'What is the condition of road_.pt'
'What is the condition of the road in this image_.pt'
'What is the overall condition of the given image_.pt'


In [ ]:
!du -ah /content/drive/MyDrive/floodnet_features/Images/Train_Image/10820.pt

129K	/content/drive/MyDrive/floodnet_features/Images/Train_Image/10820.pt


In [ ]:
# feats = torch.load("/content/10165.pt")
# feats.shape

torch.Size([1, 2048])

In [ ]:
!rm -rf /content/drive/MyDrive/floodnet_features/Images/Train_Image/*.pt

In [ ]:
# train_ds, val_ds = get_datasets(batch_size=16)

# for i in train_ds:
#     print(i)
#     break

In [ ]:
# i[1]["input_ids"].shape

In [ ]:
# def accuracy(true, pred):
#     acc = np.sum((true == pred.argmax(-1)).astype(np.float32))
#     return float(100 * acc / len(true))

# class VQAModel(nn.Module):
#     def __init__(self):
#         super().__init__()
#         self.resnet = timm.create_model('resnet50', pretrained=True, num_classes=0, global_pool='avg')
#         self.bert = BertModel.from_pretrained("bert-base-uncased")
        
#         self.device = torch.device(
#             "cuda") if torch.cuda.is_available() else torch.device("cpu")

#         self.lin = nn.Linear(2048+768, 56)
#         self.softmax = nn.Softmax(dim=-1)
#         self.push_all_to_device()
#         self.optimizer = torch.optim.Adam(self.parameters())
#         self.criterion = nn.CrossEntropyLoss()
            
#     def push_all_to_device(self):
#         """Loads all layers to GPU."""
#         self.bert = self.bert.to(self.device)
#         self.resnet = self.resnet.to(self.device)

#         self.lin = self.lin.to(self.device)
#         self.softmax = self.softmax.to(self.device)

#     def push_batch_to_device(self, batch):
#         batch[1] = {key:elem.to(self.device) for key,elem in batch[1].items()}
#         batch[0] = batch[0].to(self.device)
#         batch[2] = batch[2].to(self.device)
#         return batch
    
#     def forward(self, batch):
#         question_emb = self.bert(**batch[1])[1]  # (batch_size, hidden_size)

#         image_emb = self.resnet(batch[0])

#         x = torch.concat([question_emb, image_emb], dim=-1)

#         ans = self.lin(x)
#         ans = self.softmax(ans)
#         # print(ans)
#         # print(batch[2])
#         return ans
    
#     def train_epoch(self, train_ds, progress_bar):
#         self.train()
#         epoch_loss = []
#         epoch_acc = []
#         for batchnum, batch in enumerate(train_ds):
#             batch = self.push_batch_to_device(batch)

#             outputs = self(batch)
#             #                   (y_true, y_pred, criteria)
#             loss = self.criterion(outputs, batch[2])
#             self.optimizer.zero_grad()
#             loss.backward()

#             self.optimizer.step()

#             acc = accuracy(batch[2].detach().cpu().numpy(), outputs.detach().cpu().numpy())
#             loss_ = loss.detach().cpu().numpy()

#             # record metrics
#             epoch_loss.append(loss_)
#             epoch_acc.append(acc)

#             # progress bar
#             progress_bar.set_postfix(loss=loss_, accuracy=acc)
#             progress_bar.update(1)
#             progress_bar.set_postfix(loss=np.mean(epoch_loss),
#                                      accuracy=np.mean(epoch_acc))

#         return np.mean(epoch_loss), np.mean(epoch_acc)


#     def inference_epoch(self, val_ds):
#         """Validation loop. val DS has exactly one batch."""
#         val_epoch_loss = []
#         val_epoch_acc = []
#         val_epoch_f1 = []
#         self.eval()
#         with torch.no_grad():
#             for val_batch in val_ds:
#                 val_batch = self.push_batch_to_device(val_batch)

#                 val_outputs = self(val_batch)
#                 # print(val_batch[1])
#                 val_loss = self.criterion(val_outputs, val_batch[2])

#                 val_acc = accuracy(val_batch[2].detach().cpu().numpy(), val_outputs.detach().cpu().numpy())
                
#                 # self.calculate_metrics(val_outputs, val_batch)
#                 val_epoch_loss.append(val_loss.detach().cpu().numpy())
#                 val_epoch_acc.append(val_acc)
#         return np.mean(val_epoch_loss), np.mean(val_epoch_acc)

#     def fit(self, epochs=20):
#         optimizer = torch.optim.Adam(self.parameters())
#         criteria = nn.CrossEntropyLoss()
        
#         train_ds, val_ds = get_datasets(batch_size=8)

#         for epoch in range(epochs):
#             progress_bar = tqdm(range(len(train_ds)))
#             epoch_loss, epoch_acc = self.train_epoch(train_ds, progress_bar)
#             progress_bar.close()

#             val_loss, val_acc = self.inference_epoch(val_ds)

#             # validation loop
            
            
#             val_metrics = {"acc": val_acc, "loss": val_loss}
#             print(val_metrics)            

NameError: ignored

In [ ]:
# vqa = VQAModel()
# vqa.fit()

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertModel: ['cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.dense.bias', 'cls.seq_relationship.bias', 'cls.predictions.decoder.weight', 'cls.seq_relationship.weight', 'cls.predictions.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
100%|██████████| 508/508 [22:18<00:00,  2.64s/it, accuracy=42.2, loss=3.64]


{'acc': 50.80409356725146, 'loss': 3.5475338}


100%|██████████| 508/508 [22:15<00:00,  2.63s/it, accuracy=43.2, loss=3.62]


{'acc': 50.80409356725146, 'loss': 3.5475335}


 50%|█████     | 256/508 [11:12<10:56,  2.61s/it, accuracy=42.9, loss=3.63]   

In [ ]:
# def get_train_val_splits(jsondict):
#     total = len(jsondict.keys())
#     train = int(total * 0.9)
#     train_dict = dict()
#     val_dict = dict()
#     for i in range(train):
#         train_dict[str(i)] = jsondict[str(i)]
    
#     for j in range(total - (train+1)):
#         val_dict[str(j)] = jsondict[str(j)]
    
#     return train_dict, val_dict

# class FloodNetVQADataset(Dataset):
#     def __init__(self, images_root_path, jsondict):
#         super().__init__()

#         self.json_dict = jsondict
#         self.images_root_path = images_root_path

#         self.device = torch.device(
#             "cuda") if torch.cuda.is_available() else torch.device("cpu")

#         self.answer_mapping = json.load(open('/content/drive/MyDrive/floodnet_label_mapping.json', "r"))

#         self.tokenizer = BertTokenizer.from_pretrained("bert-base-uncased",
#                                                        do_lower_case=True)

#     def __len__(self):
#         return len(self.json_dict.keys())

#     def __getitem__(self, idx):
#         # Image, Question, Answer
#         example = self.json_dict[str(idx)]
#         imagepath = os.path.join(self.images_root_path, example["Image_ID"])
#         # print(example["Question"])
#         question = self.tokenizer(text=example["Question"],
#                                 add_special_tokens=True,
#                                 return_attention_mask=True,
#                                 max_length=50,
#                                 padding='max_length',
#                                 truncation=True,
#                                 return_tensors="pt")
#         answer = torch.tensor(self.answer_mapping.index(str(example["Ground_Truth"])))

#         # print(question["input_ids"].shape)
#         # print(question["token_type_ids"].shape)
#         # print(question["attention_mask"].shape)

#         question["input_ids"] = torch.reshape(question["input_ids"], (-1,))
#         question["token_type_ids"] = torch.reshape(question["token_type_ids"], (-1,))
#         question["attention_mask"] = torch.reshape(question["attention_mask"], (-1,))

#         transforms = T.Compose([
#             T.Resize((512,512)),
#             T.ToTensor()
#         ])

#         img = transforms(Image.open(imagepath))
        
#         return [
#             img,
#             question,
#             answer
#         ]

# def get_datasets(batch_size):
#     train_dict, val_dict = get_train_val_splits(json.load(open("/content/floodnet_data/Questions/Training Question.json", "r")))
#     train_ds = FloodNetVQADataset("/content/floodnet_data/Images/Train_Image", train_dict)
#     val_ds = FloodNetVQADataset("/content/floodnet_data/Images/Train_Image", val_dict)

#     train_dl = DataLoader(train_ds, batch_size=batch_size, shuffle=True)
#     val_dl = DataLoader(val_ds, batch_size=batch_size, shuffle=False)

#     return train_dl, val_dl

In [ ]:
N# def accuracy(true, pred):
#     acc = np.sum((true == pred.argmax(-1)).astype(np.float32))
#     return float(100 * acc / len(true))

# class VQAModel(nn.Module):
#     def __init__(self):
#         super().__init__()
#         self.resnet = timm.create_model('resnet50', pretrained=True, num_classes=0, global_pool='avg')
#         self.bert = BertModel.from_pretrained("bert-base-uncased")
        
#         self.device = torch.device(
#             "cuda") if torch.cuda.is_available() else torch.device("cpu")

#         self.lin = nn.Linear(2048+768, 56)
#         self.softmax = nn.Softmax(dim=-1)
#         self.push_all_to_device()

#     def forward(self, batch):
#         """Mandatory forward method"""

#         question_emb = self.bert(**batch[1])[1]  # (batch_size, hidden_size)

#         image_emb = self.resnet(batch[0])

#         x = torch.concat([question_emb, image_emb], dim=-1)

#         ans = self.lin(x)
#         ans = self.softmax(ans)

#         return ans

#     ### Utilities
#     def push_all_to_device(self):
#         """Loads all layers to GPU."""
#         self.bert = self.bert.to(self.device)
#         self.resnet = self.resnet.to(self.device)

#         self.lin = self.lin.to(self.device)
#         self.softmax = self.softmax.to(self.device)
    
#     def loss_fn(self, outputs, batch, criterion):
#         """Loss function. Currently only calculated loss for emotions."""
#         loss = criterion(outputs, batch[2])
#         return loss

#     def calculate_metrics(self, outputs, batch):
#         np_outputs = outputs.detach().cpu().numpy()
#         np_batch = batch[2].detach().cpu().numpy()
#         return accuracy(np_batch, np_outputs)

#     def push_batch_to_device(self, batch):
#         batch[1] = {key:elem.to(self.device) for key,elem in batch[1].items()}
#         batch[0] = batch[0].to(self.device)
#         batch[2] = batch[2].to(self.device)
#         return batch

#     def train_epoch(self, train_ds, optimizer, criteria, progress_bar):
#         self.train()
#         epoch_loss = []
#         epoch_acc = []
#         for batchnum, batch in enumerate(train_ds):
#             batch = self.push_batch_to_device(batch)

#             outputs = self(batch)
#             #                   (y_true, y_pred, criteria)
#             loss = self.loss_fn(batch, outputs, criteria)
#             optimizer.zero_grad()
#             loss.backward()

#             optimizer.step()

#             acc = self.calculate_metrics(outputs, batch)
#             loss_ = loss.detach().cpu().numpy()

#             # record metrics
#             epoch_loss.append(loss_)
#             epoch_acc.append(acc)

#             # progress bar
#             progress_bar.set_postfix(loss=loss_, accuracy=acc)
#             progress_bar.update(1)
#             progress_bar.set_postfix(loss=np.mean(epoch_loss),
#                                      accuracy=np.mean(epoch_acc))

#         return np.mean(epoch_loss), np.mean(epoch_acc)


#     def inference_epoch(self, val_ds, criterion):
#         """Validation loop. val DS has exactly one batch."""
#         val_epoch_loss = []
#         val_epoch_acc = []
#         val_epoch_f1 = []
#         self.eval()
#         with torch.no_grad():
#             for val_batch in val_ds:
#                 val_batch = self.push_batch_to_device(val_batch)

#                 val_outputs = self(val_batch)
#                 val_loss = criterion(val_outputs, val_batch[1])

#                 val_acc = self.loss_fn(val_batch, val_outputs, criterion)
                
#                 # self.calculate_metrics(val_outputs, val_batch)
#                 val_epoch_loss.append(val_loss.detach().cpu().numpy())
#                 val_epoch_acc.append(val_acc)
#         return np.mean(val_epoch_loss), np.mean(val_epoch_acc)

#     def fit(self, epochs=20):
#         optimizer = torch.optim.Adam(self.parameters())
#         criteria = nn.CrossEntropyLoss()
        
#         train_ds, val_ds = get_datasets()

#         for epoch in range(epochs):
#             progress_bar = tqdm(range(len(train_ds)))

#             epoch_loss, epoch_acc = self.train_epoch(
#                 train_ds, optimizer, criteria, progress_bar)

#             # validation loop
#             val_loss, val_acc = self.eval_epoch(
#                 val_ds, criteria)

#             val_metrics = {"acc": val_acc, "loss": val_loss}

#             progress_bar.close()
    
#     def inference(self):
#         pass

#     def evaluate(self):
#         pass

In [ ]:
# model = VQAModel()
# model.fit()

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertModel: ['cls.predictions.transform.LayerNorm.bias', 'cls.seq_relationship.weight', 'cls.predictions.decoder.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.bias', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.weight']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).

  0%|          | 0/564 [00:00<?, ?it/s]

TypeError: ignored

In [ ]:
# Image(Flooded area) --> How many buildings are flooded --> answer
# limited number of answers (objective)
# 
# VQA --> classification problem
# CNN (for image) and Transformer (for question) --> Class
#
# Challenges:
# - Naive
# - Concating feats, which is not optimal for knowledge sharing (perhaps attention should be used)
# - Not scalable